# Developing the theme of few-shots

## Init (imports, load models...)

### Necessary imports

In [1]:
import db_connect
import vc_connect
from examples import FEW_SHOT_EXAMPLES

from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_community.vectorstores.pgvector import PGVector
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chains.sql_database import query

from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.agents import Tool
from langchain import agents

from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain.agents.agent_toolkits.sql import base

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

from langchain.globals import set_verbose
set_verbose(True)

### Load models

In [2]:
llama = Ollama(model="llama2:13b", temperature=0.25, repeat_penalty=1)
llama_embeddings = OllamaEmbeddings(model="llama2:13b", temperature=0.25, repeat_penalty=1)
llama3 = Ollama(model="llama3:8b", temperature=0.25, repeat_penalty=1)
llama3_embeddings = OllamaEmbeddings(model="llama3:8b", temperature=0.25, repeat_penalty=1)
sqlcoder_embeddings = OllamaEmbeddings(model="sqlcoder:15b", temperature=0.25, repeat_penalty=1)
sqlcoder = Ollama(model='sqlcoder:15b', temperature=0.25, repeat_penalty=1)

### Create prompt

In [3]:
prefix =\
    """# Personality 
Act as PostgreSQL expert for create a query. 

# Task
Create {top_k} correct PostgreSQL codes based on the user's question.

# Examples
"""

suffix =\
    """
# Context
Use this tables as context to create query.
{table_info}

# Format
Leave ONLY the PostgreSQL CODE in the answer.

# Start
User input: {input}
PostgreSQL code: """

tool_prefix =\
    """# Personality 
Act as PostgreSQL expert for create a query. 

# Task
Create correct PostgreSQL code based on user\'s question.

# Examples
"""

tool_suffix =\
    """
# Format
Leave ONLY the PostgreSQL CODE in the answer."""

example_prompt = PromptTemplate.from_template(
    "## Example\nUser input: {input}\nSQL query: {query}\n"
)

### Connect to DB with Readonly role

In [4]:
db = db_connect.get_db()

#### Check connection

In [5]:
db.run("select * from passenger")

"[(16, 'John'), (17, 'James'), (18, 'Poul'), (19, 'Christofer'), (20, 'Superman')]"

### Connect example selector

In [6]:
example_selector = vc_connect.get_selector(llama_embeddings, FEW_SHOT_EXAMPLES)

## Simple few-shots by using `create_sql_query_chain`

In [7]:
few_shots_prompt = FewShotPromptTemplate(
    examples=FEW_SHOT_EXAMPLES,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "table_info", "top_k"],
)

In [8]:
few_shots_chain = query.create_sql_query_chain(llm=llama, db=db, prompt=few_shots_prompt)

In [9]:
res = few_shots_chain.invoke({"question": "Print the names of all airlines"})
print(res)

SELECT DISTINCT c."company_name" FROM "company" c;


In [10]:
db.run("SELECT DISTINCT c.\"company_name\" FROM \"company\" c;")

"[('American Airlines',)]"

In [80]:
res = few_shots_chain.invoke({"question": "Print the names of people that end in \"man\""})
print(res)

SELECT p."name" FROM "passenger" p WHERE p."name" LIKE '%man';


In [81]:
db.run("SELECT p.\"name\" FROM \"passenger\" p WHERE p.\"name\" LIKE \'%man\';")

ProgrammingError: (psycopg2.errors.UndefinedColumn) ОШИБКА:  столбец p.name не существует
LINE 1: SELECT p."name" FROM "passenger" p WHERE p."name" LIKE '%man...
               ^

[SQL: SELECT p."name" FROM "passenger" p WHERE p."name" LIKE '%%man';]
(Background on this error at: https://sqlalche.me/e/20/f405)

## Few-shots with `SemanticSimilarityExampleSelector`

### Connect to another DB to create embeddings

In [82]:
few_shots_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "table_info", "top_k"],
)

In [83]:
few_shots_selector_chain = query.create_sql_query_chain(llm=llama, db=db, prompt=few_shots_prompt)

In [61]:
res = few_shots_selector_chain.invoke({
    "question": "Select all passengers from the database for all time", 
    "top_k": "1"
})
print(res)

SELECT * FROM passenger;


In [62]:
db.run("SELECT * FROM passenger;")

"[(16, 'John'), (17, 'James'), (18, 'Poul'), (19, 'Christofer'), (20, 'Superman')]"

In [54]:
res = few_shots_selector_chain.invoke({
    "question": "Print the names of all airlines", 
    "top_k": "1"
})
print(res)

SELECT DISTINCT company_name FROM company;


In [58]:
db.run("SELECT DISTINCT company_name FROM company;")

"[('American Airlines',)]"

In [56]:
res = few_shots_selector_chain.invoke({
    "question": "Print the names of people that end in \"man\"", 
    "top_k": "1"
})
print(res)

SELECT passenger_name
FROM passenger
WHERE passenger_name LIKE '%man';


In [22]:
db.run("SELECT passenger_name FROM passenger WHERE passenger_name LIKE \'%man\';")

"[('Superman',)]"

## Few-shots selector with SQLCoder

In [23]:
few_shots_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "table_info", "top_k"],
)

In [24]:
few_shots_selector_chain = query.create_sql_query_chain(llm=sqlcoder, db=db, prompt=few_shots_prompt)

In [25]:
res = few_shots_selector_chain.invoke({
    "question": "Select the names of all the people who are in the airline database", 
    "top_k": "1"
})
print(res)

SELECT p.passenger_name FROM passenger p;
<|endoftext|>


In [26]:
db.run("SELECT p.passenger_name FROM passenger p;")

"[('Poul',)]"

In [27]:
res = few_shots_selector_chain.invoke({
    "question": "Print the names of all airlines", 
    "top_k": "1"
})
print(res)

SELECT DISTINCT c.company_name
FROM   company c;
<|endoftext|>


In [28]:
db.run("SELECT DISTINCT c.company_name FROM company c;")

"[('American Airlines',)]"

In [29]:
res = few_shots_selector_chain.invoke({
    "question": "Print the names of people that end in \"man\"", 
    "top_k": "1"
})
print(res)

SELECT passenger_name FROM passenger WHERE passenger_name LIKE '%man';
<|endoftext|>


In [30]:
db.run("SELECT passenger_name FROM passenger WHERE passenger_name LIKE \'%man\';")

"[('Superman',)]"

## My own agent

### Create prompt

In [31]:
template =\
"""# Personality 
Act as SQL expert for answer to an office employee. 

# Task
Create and run SQL query to answer user\'s question. 

# Tools
Always keep in mind the tools that you can use: {tools}

# Format
Give final answer in following format:
User\'s input: \"<input>\"
Created SQL query: \"<sql_code>\"
Run query result: \"<sql_run_result>\"
Final answer: \"<final_answer>\"

# Tonality
Be strict, don't use emojis. Be careful and paranoid, who checks everything 3 times

# Atypical cases 
If there is not enough information from the database to answer, write \"I cannot answer\"

Let's start a chain of thoughts!

User's input: \"{input}\"
Thought: {agent_scratchpad}
Choose the one of [{tool_names}] you need and use it"""

prompt = PromptTemplate.from_template(template)

### Define tools

#### Tool for create SQL

In [32]:
few_shots_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=tool_prefix,
    suffix=tool_suffix,
    input_variables=["input"],
)

tool_chain = (
    {"input": RunnablePassthrough()} 
    | few_shots_prompt 
    | sqlcoder 
)

sql_creator = Tool(
    name="sql_creator",
    func=tool_chain.invoke,
    description="Input to this tool an user input. Output of this tool is SQL code. Use this tool to create an SQL code based on user\'s input."
)

In [33]:
few_shots_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "table_info", "top_k"],
)

In [34]:
few_shots_selector_chain = query.create_sql_query_chain(llm=sqlcoder, db=db, prompt=few_shots_prompt)

sql_creator = Tool(
    name="sql_creator",
    func=tool_chain.invoke,
    description="Input to this tool an user input. Output of this tool is SQL code. Use this tool to create an SQL code based on user\'s input."
)

In [35]:
tools = SQLDatabaseToolkit(llm=sqlcoder, db=db).get_tools() + [sql_creator]

agent = agents.create_react_agent(
    llm=llama,
    tools=tools,
    prompt=prompt,
    output_parser=StrOutputParser(),
)

agent = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
from langchain.globals import set_debug
set_debug(False)

In [37]:
res1 = agent.invoke({
    "input": "Select the names of all the people who are in the airline database",
})
print(res1)



> Entering new AgentExecutor chain...


Error in StdOutCallbackHandler.on_agent_action callback: AttributeError("'str' object has no attribute 'log'")


AttributeError: 'str' object has no attribute 'tool'

In [ ]:
res2 = agent.invoke({
    "input": "Print the names of all airlines",
})
print(res2)

KeyboardInterrupt: 